In [ ]:
%pip install opencv-python
%pip install matplotlib
%pip install pandas
%pip install numpy
%pip install scikit-learn
%pip install torch torchvision torchaudio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import os
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim

In [3]:
pd.set_option('display.colheader_justify', 'center')  # Align headers to center
pd.set_option('display.width', None)                 # Avoid line wrapping
pd.set_option('display.float_format', '{:.3f}'.format) # Consistent float formatting

# Reading Images

In [3]:
# img_filepath = 'images/test/0/1037_left.jpeg'
# img = cv2.imread(img_filepath)
# img
# img.shape
# plt.imshow(img)

# gray_img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# plt.imshow(gray_img)

# resized_img = cv2.resize(gray_img, (320, 192))
# plt.imshow(resized_img)

# img_flatten = resized_img.flatten()
# img_flatten

# Define a Tunable Classical Model

In [8]:
image_sizes = [16, 32, 64, 128, 256, 512]

models = {
    "DecisionTree": DecisionTreeClassifier(),
    "RandomForest": RandomForestClassifier()
}

color_spaces = {
    "GRAY": cv2.COLOR_BGR2GRAY,
    "LAB": cv2.COLOR_BGR2LAB,
    "HSV": cv2.COLOR_BGR2RGB,
}

results = []

def TunableModel(img_size, model_type, model_name, color_space, color_name):
    img1D = []
    labels = []

    # Get images and convert to 1D
    
    base_path = os.path.join(os.getcwd(), 'images/Tests')
    
    for dir in os.listdir(base_path):
        dir_path = os.path.join(base_path, dir)
        for img_path in os.listdir(dir_path):
            img = cv2.imread(os.path.join(os.getcwd(), 'images/Tests', dir, img_path))
            
            # Preprocessing color and size
            gray_img = cv2.cvtColor(img, color_space)
            
            blurred_img = cv2.GaussianBlur(gray_img, (5, 5), 0)
            
            edges = cv2.Canny(blurred_img, threshold1=50, threshold2=150)
            resized_img = cv2.resize(edges, (img_size, img_size)).flatten()
            normalized_img = resized_img / 255.0
            
            img1D.append(normalized_img)
            labels.append(dir)
            
            print(f'{img_size}x{img_size} {color_name} {model_name}. Finished reading {img_path}.                                  ', end='\r')

    print(f'{img_size}x{img_size} {color_name} {model_name}. Importing to img_df.                                  ', end='\r')
    
    img_df = pd.DataFrame(img1D)
    img_df['label'] = labels

    X = img_df.drop(columns=['label'])
    y = img_df['label']

    # Split training and testing data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.1, random_state=0, stratify=y
    )
    
    print(f'{img_size}x{img_size} {color_name} {model_name}. Training the model.                                  ', end='\r')

    # Initialize the model
    model = model_type
    model.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    results.append({
        "Image_Size": img_size,
        "Color_Space": color_name,
        "Model": model_name,
        "Accuracy": accuracy,
        "Precision": precision,
        "Recall": recall,
        "F1_Score": f1
    })

for img_size in image_sizes:
    for model_name, model_type in models.items():
        for color_name, color_space in color_spaces.items():
            TunableModel(img_size, model_type, model_name, color_space, color_name)

results_df = pd.DataFrame(results)

# Sort the DataFrame by 'F1_Score' in descending order
sorted_df = results_df.sort_values(by='F1_Score', ascending=False)

# Print the sorted DataFrame
print(sorted_df)

c:\Users\Adrian De Guzman\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


c:\Users\Adrian De Guzman\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


c:\Users\Adrian De Guzman\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


    Image_Size Color_Space         Model  Accuracy  Precision    Recall  \                                         
9           32        GRAY  RandomForest  0.846154   0.923077  0.846154   
10          32         LAB  RandomForest  0.846154   0.923077  0.846154   
6           32        GRAY  DecisionTree  0.846154   0.923077  0.846154   
8           32         HSV  DecisionTree  0.846154   0.923077  0.846154   
29         256         HSV  RandomForest  0.846154   0.871795  0.846154   
35         512         HSV  RandomForest  0.846154   0.871795  0.846154   
34         512         LAB  RandomForest  0.846154   0.897436  0.846154   
33         512        GRAY  RandomForest  0.846154   0.897436  0.846154   
11          32         HSV  RandomForest  0.846154   0.871795  0.846154   
15          64        GRAY  RandomForest  0.846154   0.871795  0.846154   
17          64         HSV  RandomForest  0.846154   0.871795  0.846154   
21         128        GRAY  RandomForest  0.846154   0.8717

In [14]:
def TunableModelwithResults(img_size, model_type, model_name, color_space, color_name):
    img1D = []
    labels = []

    # Get images and convert to 1D
    
    base_path = os.path.join(os.getcwd(), 'images/Tests')
    
    for dir in os.listdir(base_path):
        dir_path = os.path.join(base_path, dir)
        for img_path in os.listdir(dir_path):
            img = cv2.imread(os.path.join(os.getcwd(), 'images/Tests', dir, img_path))
            
            # Preprocessing color and size
            gray_img = cv2.cvtColor(img, color_space)
            
            blurred_img = cv2.GaussianBlur(gray_img, (5, 5), 0)
            
            edges = cv2.Canny(blurred_img, threshold1=50, threshold2=150)
            resized_img = cv2.resize(edges, (img_size, img_size)).flatten()
            normalized_img = resized_img / 255.0
            
            img1D.append(normalized_img)
            labels.append(dir)
            
            print(f'{img_size}x{img_size} {color_name} {model_name}. Finished reading {img_path}.                                  ', end='\r')

    print(f'{img_size}x{img_size} {color_name} {model_name}. Importing to img_df.                                  ', end='\r')
    
    img_df = pd.DataFrame(img1D)
    img_df['label'] = labels

    X = img_df.drop(columns=['label'])
    y = img_df['label']

    # Split training and testing data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.1, random_state=0, stratify=y
    )
    
    print(f'{img_size}x{img_size} {color_name} {model_name}. Training the model.                                  ', end='\r')

    # Initialize the model
    model = model_type
    model.fit(X_train, y_train)

    # Predict and evaluate
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, target_names=y_test.unique()))

    # Evaluation Metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')

    print("\nEvaluation Metrics:")
    print("-" * 25)
    print(f"{'Accuracy':<15}{accuracy:>10.4f}")
    print(f"{'Precision':<15}{precision:>10.4f}")
    print(f"{'Recall':<15}{recall:>10.4f}")
    print(f"{'F1 Score':<15}{f1:>10.4f}")

In [22]:
TunableModelwithResults(32, RandomForestClassifier(), "RandomForest", cv2.COLOR_BGR2GRAY, "GRAY")

32x32 GRAY RandomForest. Training the model.                                                                     
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           2       1.00      0.67      0.80         3
           1       0.50      1.00      0.67         2
           3       1.00      0.67      0.80         3
           4       1.00      1.00      1.00         3

    accuracy                           0.85        13
   macro avg       0.90      0.87      0.85        13
weighted avg       0.92      0.85      0.86        13


Evaluation Metrics:
-------------------------
Accuracy           0.8462
Precision          0.9231
Recall             0.8462
F1 Score           0.8564


In [24]:
import joblib

model = TunableModelwithResults(32, RandomForestClassifier(), "RandomForest", cv2.COLOR_BGR2GRAY, "GRAY")

# Save the entire model (architecture + weights) using joblib
joblib.dump(model, 'Group11-ClassicalModel.joblib')



32x32 GRAY RandomForest. Training the model.                                                                     
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           2       1.00      0.67      0.80         3
           1       0.50      1.00      0.67         2
           3       1.00      0.67      0.80         3
           4       1.00      1.00      1.00         3

    accuracy                           0.85        13
   macro avg       0.90      0.87      0.85        13
weighted avg       0.92      0.85      0.86        13


Evaluation Metrics:
-------------------------
Accuracy           0.8462
Precision          0.9231
Recall             0.8462
F1 Score           0.8564


['Group11-ClassicalModel.joblib']